### Notes

For Span obj:
+ build off author and publisher names
+ text
+ start char end char
+ label (author or publisher name)
+ group (make sure authors and pubs don't overlap)
+ as a group (this group's closeness to KB ids)
    + dictionary of id and then its score
    
For ReviewObj:
+ all author spans attribute
+ all pub spans attribute

Also refer to spaCy spans -> https://spacy.io/api/span
+ doc (doc object)
+ start (int)
+ end (int)
+ label (int/unicode)
+ kb_id (int/unicode)
+ vector (numpy.ndarray[ndim=1, dtype='float32'])

In [1]:
import sys
sys.path.append('../')

In [2]:
from application.name_obj_classes import PubName, PersonName
from application.review_obj_class import ReviewObj
import pandas as pd
import os
%pprint

Pretty printing has been turned OFF


In [3]:
# loading in files
directory = "../../aps_reviews_50/aps_reviews/"
filenames = os.listdir(directory)
txts = []
for file in filenames:
    with open(directory + file) as f:
        txts.append(f.read())

In [4]:
review_list = []
for filename, txt in zip([x.split('.')[0] for x in filenames], txts):
    review_list.append(ReviewObj(filename, txt))

Span object will contain:
+ Review ID (parent doc)
+ Start char (int)
+ End char (int)
+ Label (person/publisher)
+ Group (int)
+ Name variations
+ VIAF but not yet

In [9]:
# review id
review_list[0].person_names[0].review_id

'136726613'

In [10]:
# start and end char
review_list[0].person_names[0].review_loc_chars

(95, 120)

In [8]:
# label
review_list[0].person_names[0].name_type

'person'

In [13]:
review_list[0].person_names[0].getNameVariants()

['c dudley warner', 'c warner', 'c d warner', 'charles dudley warner', 'warner', 'charles d warner', 'charles warner']

In [ ]:
class NameSpan():
    

In [ ]:
def consolidatePersonNames(name_list):
    """
    Takes list of NameSpan objects
    """
    name_set = []
    used_indices = []
    last_names = sorted([name.last_name for name in name_list], key=len)
    
    for i, name in enumerate(last_names):
        if i not in used_indices:
            full_name = [x for x in name_list if x.last_name == name][0]
            name_holder = [full_name]
            for j, name2 in enumerate(last_names):
                full_name2 = [x for x in name_list if x.last_name == name2][0]
                if (i < (len(last_names) - 1)) and (i!=j):
                    if (edit_distance(name, name2[:len(name)+1]) < 2) and (j not in used_indices):
                        if (full_name.first_initial==full_name2.first_initial and full_name.middle_initial==full_name2.middle_initial or full_name.title==full_name2.title) or (full_name.first_initial==full_name2.first_initial or full_name.middle_initial==full_name2.middle_initial and full_name.title==full_name2.title):
                            name_holder.append(full_name2)
                            used_indices.append(j)
            used_indices.append(i)
            name_set.append(name_holder)
    
    return name_set  